## 2日目
* 関手きた！！！！！！！！

### Functor
* Functorは全体を写せるもの(map over)の型クラス?

scalaz.Functorの実装

```scala
trait Functor[F[_]] { self =>
    /** List `f` into `F` and apply to `F[A]`. */
    def map[A, B](fa: F[A])(f: A => B): F[B]
    
    ...
}
```

これが可能とする演算子

```scala
trait FunctorOps[F[_],A] extends Ops[F[A]] {
    implicit def F: Functor[F]
    ///
    import Leibniz.===
    
    final def map[B](f: A => B): F[B] = F.map(self)(f)
    
    ...
}
```

In [1]:
import scalaz._, Scalaz._

import scalaz._, Scalaz._

In [2]:
List(1, 2, 3) map {_ + 1}

res1: List[Int] = List(2, 3, 4)

* なんだ関手ってつまりmapの事なのかー

In [3]:
(1, 2, 3) map {_ + 1}

res2: (Int, Int, Int) = (1, 2, 4)

* Tupleは最後の値のみに適用される
* これは Tuple[A, B, C] に対して適用しているから
* だから Tupleならこれもエラーなく動くはず

In [4]:
("a", "b", 1) map {_ + 1}

res3: (String, String, Int) = ("a", "b", 2)

### Functorとしての関数

In [5]:
((x: Int) => x + 1) map {_ * 7}

res4: Int => Int = <function1>

In [6]:
res4(3)

res5: Int = 28

Haskelの例

```
ghci> fmap (*3) (+100) 1
303
```

In [7]:
(((_: Int) * 3) map {_ + 100}) (1)

res6: Int = 103

In [8]:
Functor[List].lift {(_: Int) * 2}

res7: List[Int] => List[Int] = <function1>

In [9]:
res7(List(3))

res8: List[Int] = List(6)

In [10]:
List(1, 2, 3) >| "x"

res9: List[String] = List("x", "x", "x")

In [11]:
List(1, 2, 3) as "x"

res10: List[String] = List("x", "x", "x")

In [12]:
List(1, 2, 3).fpair

res11: List[(Int, Int)] = List((1, 1), (2, 2), (3, 3))

In [13]:
List(1, 2, 3).strengthL("x")

res12: List[(String, Int)] = List(("x", 1), ("x", 2), ("x", 3))

In [14]:
List(1, 2, 3).strengthR("x")

res13: List[(Int, String)] = List((1, "x"), (2, "x"), (3, "x"))

In [15]:
List(1, 2, 3).void

res14: List[Unit] = List((), (), ())

* Haskellとは関数適用の順番が違うので気をつける
* その深い意味まではよくわからんかったけど今はそこまで理解できなくていいかな

### Applicative

In [15]:
List(1, 2, 3, 4) map {(_: Int) * (_: Int)}

cmd15.sc:1: type mismatch;
 found   : (Int, Int) => Int
 required: Int => ?
val res15 = List(1, 2, 3, 4) map {(_: Int) * (_: Int)}
                                           ^

: 

In [16]:
List(1, 2, 3, 4) map {(_: Int) * (_: Int)}.curried

res15: List[Int => Int] = List(<function1>, <function1>, <function1>, <function1>)

In [17]:
res15 map {_(9)}

res16: List[Int] = List(9, 18, 27, 36)

* curriedメソッドでカリー化できるんだ。便利ー

scalaz の Applicative

```scala
trait Applicative[F[_]] extends Apply[F] { self =>
    def point[A](a: => A): F[A]
    
    /** alias for `point` */
    def pure[A](a: => A): F[A] = point(a)
    
    ...
}
```

In [18]:
1.point[List]

res17: List[Int] = List(1)

In [19]:
1.point[Option]

res18: Option[Int] = Some(1)

In [20]:
1.point[Option] map {_ + 2}

res19: Option[Int] = Some(3)

In [21]:
1.point[List] map {_ + 2}

res20: List[Int] = List(3)

### Apply
* fmapは普通の関数とファンクター値を引数にとって、関数をファンクター値の中の値に適用する
* <*>は関数の入っているファンクター値と値の入っているファンクター値を引数にとって、1つ目のファンクターの中身である関数を2つ目のファンクターの中身に適用する

In [22]:
trait Apply[F[_]] extends Functor[F] { self =>
    def ap[A, B](fa: => F[A])(f: => F[A => B]): F[B]
}

defined trait Apply

In [23]:
9.some <*> {(_: Int) + 3}.some

res22: Option[Int] = Some(12)

In [24]:
1.some <* 2.some

res23: Option[Int] = Some(1)

In [25]:
none <* 2.some

res24: Option[Nothing] = None

In [26]:
1.some *> 2.some

res25: Option[Int] = Some(2)

In [27]:
none *> 2.some

res26: Option[Int] = None

In [28]:
9.some <*> {(_: Int) + 3}.some

res27: Option[Int] = Some(12)

In [29]:
3.some <*> { 9.some <*> {(_: Int) + (_: Int)}.curried.some }

res28: Option[Int] = Some(12)

* 例から何が嬉しいのかよくわからん

### Applicative Style

In [30]:
^(3.some, 5.some) {_ + _}

res29: Option[Int] = Some(8)

In [31]:
^(3.some, none[Int]) {_ + _}

res30: Option[Int] = None

In [32]:
(3.some |@| 5.some) {_ + _}

res31: Option[Int] = Some(8)

### ApplyとしてのList

In [33]:
List(1, 2, 3) <*> List((_: Int) * 0, (_: Int) + 100, (x: Int) => x * x)

res32: List[Int] = List(0, 0, 0, 101, 102, 103, 1, 4, 9)

In [34]:
List(3, 4) <*> { List(1, 2) <*> List({(_: Int) + (_: Int)}.curried, {(_: Int) * (_: Int)}.curried) }

res33: List[Int] = List(4, 5, 5, 6, 3, 4, 6, 8)

In [35]:
(List("ha", "heh", "hmm") |@| List("?", "!", ".")) {_ + _}

res34: List[String] = List("ha?", "ha!", "ha.", "heh?", "heh!", "heh.", "hmm?", "hmm!", "hmm.")

### Zip List

In [36]:
streamZipApplicative.ap(Tags.Zip(Stream(1, 2))) (Tags.Zip(Stream({(_: Int) + 3}, {(_: Int) * 2})))

res35: {type Tag = scalaz.Tags.Zip;type Self = scala.collection.immutable.Stream[Int]} = Stream(4, ?)

In [36]:
res35.toList

cmd36.sc:1: value toList is not a member of Object{type Tag = scalaz.Tags.Zip; type Self = scala.collection.immutable.Stream[Int]}
val res36 = res35.toList
                  ^

: 

* 動かなきゃいけないはずだけど、もはや何でエラー出てるのかわからん

### Applicativeの便利な関数
* Haskell: Control.ApplicativeにはliftA2という関数がある

```haskell
liftA2 :: (Applicative f) => (a -> b -> c) -> f a -> f b -> f c .
```

In [2]:
Apply[Option].lift2((_: Int) :: (_: List[Int]))

res1: (Option[Int], Option[List[Int]]) => Option[List[Int]] = <function2>

In [3]:
res1(3.some, List(4).some)

res2: Option[List[Int]] = Some(List(3, 4))

In [4]:
def sequenceA[F[_]: Applicative, A](list: List[F[A]]): F[List[A]] = list match {
    case Nil => (Nil: List[A]).point[F]
    case x :: xs => (x |@| sequenceA(xs)) {_ :: _}
}

defined function sequenceA

In [5]:
sequenceA(List(1.some, 2.some))

res4: Option[List[Int]] = Some(List(1, 2))

In [6]:
sequenceA(List(3.some, none, 1.some))

res5: Option[List[Int]] = None

In [7]:
sequenceA(List(List(1, 2, 3), List(4, 5, 6)))

res6: List[List[Int]] = List(
  List(1, 4),
  List(1, 5),
  List(1, 6),
  List(2, 4),
  List(2, 5),
  List(2, 6),
  List(3, 4),
  List(3, 5),
  List(3, 6)
)

In [8]:
type Function1Int[A] = ({type l[A]=Function1[Int, A]})#l[A]

defined type Function1Int

In [9]:
sequenceA(List((_: Int) + 3, (_: Int) + 2, (_: Int) + 1): List[Function1Int[Int]])


res8: Int => List[Int] = <function1>

In [10]:
res8(3)

res9: List[Int] = List(6, 5, 4)

* こんな意味わからんコード出てきたら読めない自信ある
* scalazを取り入れるとチームがギスギスするというのもわかる気がする
* 昨日Scalaの型クラスと比較していると思っていたところは実はHaskellとの比較だった。通りでScalaで調べてもBoundedとか見つからないはず